In [ ]:
# if running in Colab run this first, otherwise make sure flair[word-embeddings] is installed
import torch
!pip install flair[word-embeddings]
import flair
# it looks like we have to restart the Colab runtime after installation

In [ ]:
import os
from pathlib import Path

from flair.data import Corpus, Sentence
from flair.datasets import ColumnCorpus
from flair.embeddings import WordEmbeddings, FlairEmbeddings, StackedEmbeddings, CharacterEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer


if __name__ == "__main__":
    # load corpus
    columns = {0: 'text', 1: 'ner'}
    data_folder = '/home/dataset' # upload dataset to this folder in Colab
    corpus: Corpus = ColumnCorpus(data_folder, columns,
                                train_file='train.txt',
                                test_file='test.txt',
                                dev_file='dev.txt',
                                column_delimiter='\t')

    # extract the labels from the corpus
    label_type = 'ner'
    label_dict = corpus.make_label_dictionary(label_type=label_type, add_unk=False)

    # train model
    embedding_types = [
        WordEmbeddings('twitter'),
        FlairEmbeddings('news-forward'),
        FlairEmbeddings('news-backward')
    ]

    embeddings = StackedEmbeddings(embeddings=embedding_types)

    tagger = SequenceTagger(hidden_size=256,
                            embeddings=embeddings,
                            tag_dictionary=label_dict,
                            tag_type=label_type,
                            tag_format="BIO")

    trainer = ModelTrainer(tagger, corpus)

    path = '/home/model/' # model is saved here
    trainer.train(path,
                    learning_rate=0.05,
                    mini_batch_size=32,
                    max_epochs=25)
    
    # tuning learning rate - all produced lower scores than 0.05
    learning_rates = [0.01, 0.005, 0.0025, 0.001]
    for lr in learning_rates:
        path = '/home/model-lr-' + str(lr)
        trainer.train(path,
                      learning_rate=lr,
                      mini_batch_size=32,
                      max_epochs=25)